In [1]:
import duckdb
import os.path

import ipaddress
import netaddr

FLUTTER_PARQUET = os.path.expanduser("~/src/ripe/rpki/rpki-flutter/data/*.parquet")

In [2]:
tbl = duckdb.read_parquet(FLUTTER_PARQUET)

In [3]:
def prefix_first_to_bytes(inp: str) -> bytes:
    range = netaddr.IPNetwork(inp)
    buf = ipaddress.ip_address(range.first).packed
    to_pad = len(buf) - (128//8)

    return b'\x00' * to_pad + buf

def prefix_last_to_bytes(inp: str) -> bytes:
    range = netaddr.IPNetwork(inp)
    buf = ipaddress.ip_address(range.last).packed
    to_pad = len(buf) - (128//8)

    return b'\x00' * to_pad + buf

duckdb.create_function("prefix_first", prefix_first_to_bytes)
duckdb.create_function("prefix_last", prefix_last_to_bytes)


/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_22160/1647169654.py:15: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  duckdb.create_function("prefix_first", prefix_first_to_bytes)


In [4]:
duckdb.query("select prefix_first(pfx) from (select * from '" + FLUTTER_PARQUET + "' where pfx = '1.0.0.0/24')")

┌───────────────────┐
│ prefix_first(pfx) │
│       blob        │
├───────────────────┤
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│        ·          │
│        ·          │
│        ·          │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
├───────────────────┤
│     124 rows      │
│    (20 shown)     │
└───────────────────┘